In [1]:
import pandas as pd
import numpy as np
import graphlab as gl
from datetime import datetime
from operator import itemgetter
from collections import Counter
from functools import partial

import gensim
from gensim import corpora

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()

C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
main_path = "D:\Grad Notes\Search\Project\Data\yelp_dataset\dataset\\"

In [3]:
## Stopword list 1
stop = stopwords.words('english')
len(stop)

## Stop list 2
f = open(main_path+'stopwords.txt','r')
stop2 = f.readlines()[0].split("\r")

stop_all = stop + stop2
stop_all = set(stop_all)

In [4]:
## Read business dataset
df_bus = pd.read_csv(main_path+"business.csv")
df_bus.head()

,attributes.Ambience.divey,attributes.RestaurantsDelivery,attributes.DogsAllowed,postal_code,hours.Thursday,attributes.HairSpecializesIn.coloring,attributes.BestNights.sunday,attributes.BYOB,attributes.AgesAllowed,attributes.Music.video,...,attributes.Caters,attributes.RestaurantsReservations,attributes.DietaryRestrictions.dairy-free,attributes.DietaryRestrictions.vegan,attributes.Ambience.romantic,attributes.Music.jukebox,attributes.Ambience.upscale,attributes.RestaurantsTakeOut,attributes.BikeParking,attributes.OutdoorSeating
0,NaN,NaN,NaN,44143,10:00-21:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN
1,False,True,NaN,28215,10:00-22:00,NaN,NaN,NaN,NaN,NaN,...,NaN,False,NaN,NaN,False,NaN,False,True,NaN,False
2,NaN,NaN,NaN,M4K 1N7,10:00-19:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False
3,NaN,NaN,NaN,85258,9:00-17:00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,False,NaN,85016,NaN,NaN,NaN,NaN,NaN,NaN,...,True,False,NaN,NaN,False,NaN,False,True,True,False


In [5]:
## Get location info
df_bus_loc = df_bus[['business_id','postal_code','latitude','longitude']]
df_bus_loc.to_csv(main_path+'business_loc.csv')

In [6]:
#For Las Vegas
df_bus_las = df_bus[(df_bus['latitude'] > 36.133583) & (df_bus['latitude'] < 36.383558) & \
                    (df_bus['longitude'] > -115.418743) & (df_bus['longitude'] < -115.059459)]
df_bus_las.shape

df_las_sub = df_bus_las[['business_id','categories']]
df_las_sub.drop_duplicates(inplace=True)
df_las_sub.size

las_id = df_bus_las['business_id'].tolist()
print len(las_id)

11956


C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


In [7]:
#For Arizona
df_bus_phx = df_bus[(df_bus['latitude'] > 33.303001) & (df_bus['latitude'] < 33.918307) & \
                    (df_bus['longitude'] > -112.344998) & (df_bus['longitude'] < -111.923398)]
df_bus_phx.shape

df_phx_sub = df_bus_phx[['business_id','categories']]
df_phx_sub.drop_duplicates(inplace=True)
df_phx_sub.size

phx_id = df_bus_phx['business_id'].tolist()
print len(phx_id)

28085


## Working with the Las Vegas dataset

In [8]:
## Filtering businesses with the most common category tags

df_las_sub = df_las_sub[df_las_sub['categories']!='[]']

cat_count = Counter(df_las_sub['categories'])
cat_count.most_common(10)

[("[u'Restaurants', u'Mexican']", 99),
 ("[u'Mexican', u'Restaurants']", 84),
 ("[u'Beauty & Spas', u'Nail Salons']", 58),
 ("[u'Pizza', u'Restaurants']", 58),
 ("[u'Restaurants', u'Pizza']", 57),
 ("[u'Nail Salons', u'Beauty & Spas']", 55),
 ("[u'Automotive', u'Auto Repair']", 47),
 ("[u'Financial Services', u'Banks & Credit Unions']", 44),
 ("[u'Food', u'Coffee & Tea']", 43),
 ("[u'Hair Salons', u'Beauty & Spas']", 43)]

In [9]:
## Obtain the top 100 categories

cat_list = cat_count.most_common(100)
cat_list = [i[0] for i in cat_list]
cat_list[:5]

["[u'Restaurants', u'Mexican']",
 "[u'Mexican', u'Restaurants']",
 "[u'Beauty & Spas', u'Nail Salons']",
 "[u'Pizza', u'Restaurants']",
 "[u'Restaurants', u'Pizza']"]

In [10]:
## Get the user reviews dataset
df_rev = pd.read_csv(main_path+"review.csv")
df_rev.shape

(4736897, 9)

In [11]:
df_rev.head()

,funny,user_id,review_id,text,business_id,stars,date,useful,cool
0,0,cjpdDjZyprfyDG3RlkVG3w,VfBHSwC5Vz_pbFluy07i9Q,My girlfriend and I stayed here for 3 nights a...,uYHaNptLzDLoV_JZ_MuzUA,5,2016-07-12,0,0
1,0,bjTcT8Ty4cJZhEOEo01FGA,3zRpneRKDsOPq92tq7ybAA,If you need an inexpensive place to stay for a...,uYHaNptLzDLoV_JZ_MuzUA,3,2016-10-02,0,0
2,0,AXgRULmWcME7J6Ix3I--ww,ne5WhI1jUFOcRn-b-gAzHA,Mittlerweile gibt es in Edinburgh zwei Ableger...,uYHaNptLzDLoV_JZ_MuzUA,3,2015-09-17,0,0
3,0,oU2SSOmsp_A8JYI7Z2JJ5w,llmdwOgDReucVoWEry61Lw,Location is everything and this hotel has it! ...,uYHaNptLzDLoV_JZ_MuzUA,4,2016-08-21,0,0
4,0,0xtbPEna2Kei11vsU-U2Mw,DuffS87NaSMDmIfluvT83g,gute lage im stadtzentrum. shoppingmeile und s...,uYHaNptLzDLoV_JZ_MuzUA,5,2013-11-20,0,0


In [12]:
## Get reviews specific to Las Vegas

df_rev_las = df_rev[df_rev['business_id'].isin(las_id)]
df_rev_las.head()

,funny,user_id,review_id,text,business_id,stars,date,useful,cool
482,0,jgzD7eBwZrasqy6wUy122w,BLIJFaJZ-_fOcBs16fL_6g,"Loved, Loved, Loved. It is a simple place, but...",OQcvO5P3gH0cuJ-bPXwfQQ,5,2017-04-01,0,0
483,0,2v_meK453YAWXz4NjJ9abA,VuKbGklNbOESJSx76_EjyA,Is a small restaurant food is good! Also the o...,OQcvO5P3gH0cuJ-bPXwfQQ,5,2016-11-05,0,0
484,0,cdFWtOgA1PAkNYkiwzUJbQ,HBaAmcS9zp5rY1qiMuWygA,Best Mexican restaurant in Vegas. Meat is supe...,OQcvO5P3gH0cuJ-bPXwfQQ,5,2017-06-01,1,0
485,0,60ENRcuFKNf9oeA2zWROFQ,41ORR2OPi2CP0FAoiWjwWA,I would've like to take a before picture when...,OQcvO5P3gH0cuJ-bPXwfQQ,5,2017-07-16,1,0
486,42,Fv0e9RIV9jw5TX3ctA1WbA,Oxz26pqpIb7dDVeuUzNZlg,We had originally planned on eating at the Sin...,OQcvO5P3gH0cuJ-bPXwfQQ,4,2017-06-20,66,67


In [13]:
df_rev_las.shape

(458310, 9)

In [14]:
df_rev_las = df_rev_las.merge(df_las_sub, on='business_id', how = 'inner')
df_rev_las.shape

(457998, 10)

In [15]:
## Filter reviews belonging to 100 most frequent categories

df_rev_new = df_rev_las[df_rev_las['categories'].isin(cat_list)]
df_rev_new.shape

(57189, 10)

## Feature generation from text data

In [16]:
def clean_text(rtext):
    stop_free = " ".join([i for i in rtext.lower().split() if i not in stop_all])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    return punc_free

In [19]:
df_rev_new['cleaned_text'] = df_rev_new['text'].apply(clean_text)
df_rev_new.head(3)

C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,funny,user_id,review_id,text,business_id,stars,date,useful,cool,categories,cleaned_text
0,0,jgzD7eBwZrasqy6wUy122w,BLIJFaJZ-_fOcBs16fL_6g,"Loved, Loved, Loved. It is a simple place, but...",OQcvO5P3gH0cuJ-bPXwfQQ,5,2017-04-01,0,0,"[u'Restaurants', u'Mexican']",loved loved loved simple place dont place deco...
1,0,2v_meK453YAWXz4NjJ9abA,VuKbGklNbOESJSx76_EjyA,Is a small restaurant food is good! Also the o...,OQcvO5P3gH0cuJ-bPXwfQQ,5,2016-11-05,0,0,"[u'Restaurants', u'Mexican']",small restaurant food good owners friendly mak...
2,0,cdFWtOgA1PAkNYkiwzUJbQ,HBaAmcS9zp5rY1qiMuWygA,Best Mexican restaurant in Vegas. Meat is supe...,OQcvO5P3gH0cuJ-bPXwfQQ,5,2017-06-01,1,0,"[u'Restaurants', u'Mexican']",mexican restaurant vegas meat super soft tasty...


In [20]:
n3 = datetime.now()

sf_rev = gl.SFrame(df_rev_new[['review_id','cleaned_text']])

sf_pos = gl.SFrame()
sf_pos['nouns_tags'] = gl.text_analytics.extract_parts_of_speech(sf_rev['cleaned_text'],chosen_pos=[gl.text_analytics.PartOfSpeech.NOUN])

print "POS Tagging runtime: ",datetime.now()-n3
sf_pos

This non-commercial license of GraphLab Create for academic use is assigned to bchalasa@iu.edu and will expire on November 26, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\BHARGA~1\AppData\Local\Temp\graphlab_server_1512399292.log.0


POS Tagging runtime:  0:03:23.233000


nouns_tags
"{'NOUN': {'customer': 1L,'rating': 1L, 'decor': ..."
"{'NOUN': {'food': 1L,'owners': 1L, ..."
"{'NOUN': {'yummy': 1L,'meat': 1L, 'service': ..."
"{'NOUN': {'picture': 1L,'burrito': 1L, ..."
"{'NOUN': {'shop': 1L,'cheese': 2L, 'figure': ..."
"{'NOUN': {'service': 1L,'business': 1L, ..."
"{'NOUN': {'cheese': 1L,'terms': 1L, 'lot': 1L, ..."
"{'NOUN': {'shop': 2L,'pueblo': 1L, 'line': ..."
"{'NOUN': {'daughter': 1L,'service': 1L, 'food': ..."
"{'NOUN': {'week': 1L,'asada': 1L, 'tacos': ..."


In [21]:
df_rev_new['noun_tags'] = list(sf_pos['nouns_tags'])

C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [22]:
def lemmatize_word(word):
    return lemma.lemmatize(word.decode('utf-8'))
    
def format_tags(tags):
    nlist = tags['NOUN'].keys()
    lem_list = list(set([i for i in nlist]))
    
    return lem_list

In [23]:
df_rev_new['noun_tags'] = df_rev_new['noun_tags'].apply(format_tags)

C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [24]:
df_rev_new.head(3)

,funny,user_id,review_id,text,business_id,stars,date,useful,cool,categories,cleaned_text,noun_tags
0,0,jgzD7eBwZrasqy6wUy122w,BLIJFaJZ-_fOcBs16fL_6g,"Loved, Loved, Loved. It is a simple place, but...",OQcvO5P3gH0cuJ-bPXwfQQ,5,2017-04-01,0,0,"[u'Restaurants', u'Mexican']",loved loved loved simple place dont place deco...,"[customer, rating, decor, dicount, food, note,..."
1,0,2v_meK453YAWXz4NjJ9abA,VuKbGklNbOESJSx76_EjyA,Is a small restaurant food is good! Also the o...,OQcvO5P3gH0cuJ-bPXwfQQ,5,2016-11-05,0,0,"[u'Restaurants', u'Mexican']",small restaurant food good owners friendly mak...,"[food, owners, restaurant]"
2,0,cdFWtOgA1PAkNYkiwzUJbQ,HBaAmcS9zp5rY1qiMuWygA,Best Mexican restaurant in Vegas. Meat is supe...,OQcvO5P3gH0cuJ-bPXwfQQ,5,2017-06-01,1,0,"[u'Restaurants', u'Mexican']",mexican restaurant vegas meat super soft tasty...,"[yummy, meat, service, restaurant, fish, vegas..."


In [25]:
##Store results
df_rev_new.to_csv(main_path+'las_test_pos.csv')

In [26]:
tagged_revs = df_rev_new['noun_tags'].tolist()
tagged_revs[:3]

[['customer',
  'rating',
  'decor',
  'dicount',
  'food',
  'note',
  'compensation',
  'place'],
 ['food', 'owners', 'restaurant'],
 ['yummy', 'meat', 'service', 'restaurant', 'fish', 'vegas', 'salsa']]

## Train LDA Model

In [ ]:
n3 = datetime.now()

##Run topic modeling over all the reviews

tagged_revs = df_rev_new['noun_tags'].tolist()

 # Creating the term dictionary 
dictionary = corpora.Dictionary(tagged_revs)
dictionary.filter_extremes(keep_n=10000)
dictionary.compactify()
len(dictionary)

# Converting into Document Term Matrix
doc_term_matrix = [dictionary.doc2bow(rtxt) for rtxt in tagged_revs]

# LDA model object 
Lda = gensim.models.ldamodel.LdaModel

# Training the LDA model
ldamodel = Lda(doc_term_matrix, num_topics=50, id2word = dictionary, chunksize=3000, update_every=2, passes=2)

print "LDA Runtime: ",datetime.now() - n3

In [59]:
# lda_topics = ldamodel.print_topics(num_topics=50, num_words=10)
# lda_topics

In [60]:
#ldamodel.save(main_path+'LdaModels\las_top50')

In [27]:
## Load previously generated model

Lda = gensim.models.ldamodel.LdaModel
ldamodel = Lda.load(main_path+'LdaModels\las_top50')

In [33]:
## Infer a topic name for thr LDA topics

topic_map =[(0,'service;steak'),(1, 'fast food;mexican'),(2, 'ambience'),(3, 'location'),(4, 'italian'),(5, 'gas station;fast food'),\
            (6, 'arts;entertainment'),(7, 'home services'),(8, 'ice cream'),(9, 'management;service'),(10, 'fast food'),(11, 'arts;entertainment'),\
            (12, 'breakfast;restaurant'),(13, 'ice cream'),(14, 'pizza'), (15, 'service;experience'), (16, 'none'), (17, 'bar;nightlife'), (18, 'hair salon;beauty'), \
            (19, 'ice cream'), (20, 'mexican'), (21, 'apartment;home services'), (22, 'parks'),(23, 'location;service'), (24, 'vet;health;medical'),\
            (25, 'Asian;thai;chinese'),(26, 'car dealer;auto service;car;auto'),(27, 'banks;financial'), (28, 'pizza;italian'),\
            (29, 'none'),(30, 'apartment;management'),(31, 'service;apartments'), (32, 'apartments'),(33, 'pizza;italian'), \
            (34, 'cafe;coffee'),(35, 'cafe;coffee'),(36, 'apartment'),(37, 'service;customer'),(38, 'dentist;health'),(39, 'none'),\
            (40, 'ambience'),(41, 'church'),(42, 'italian'),(43, 'fast food;american'), (44, 'nail salon;beauty'),(45, 'hair salon;beauty'),\
            (46, 'weddings;jewelery;shopping'),(47, 'dinner;food;restaurant'),(48, 'auto service;car;auto'), (49, 'optometrist;health')]

In [34]:
tlist = [i[1] for i in topic_map]
lda_topic_map = dict(zip(range(51),tlist))

In [127]:
# len(cat_list)

# from random import randint
# x_ind =[randint(0,99) for p in range(10)]

# fin_topic_list = [cat_list[i] for i in x_ind]
# fin_topic_list

# df_sub1 = df_rev_new[df_rev_new['categories'].isin(fin_topic_list)]
# df_sub1.shape

100

## METHOD I - Predicting topics from all the concatenated topics of a business

In [35]:
## Predicting topics from all the reviews of a business
def predict_topics(bus_id):
    
    df_sub = df_rev_new[(df_rev_new['business_id'] == bus_id)]
    
    review_list = df_sub['noun_tags'].tolist()
    noun_tags = list(set([j for i in review_list for j in i]))

    new_review_bow = dictionary.doc2bow(noun_tags)
    new_review_lda = ldamodel[new_review_bow]

    return sorted(new_review_lda,key=itemgetter(1), reverse=True)[:5]


In [36]:
## Predict topics for cancatenated nouns of all business reviews

tagged_revs = df_rev_new['noun_tags'].tolist()

 # Creating the term dictionary 
dictionary = corpora.Dictionary(tagged_revs)
dictionary.filter_extremes(keep_n=10000)
dictionary.compactify()

df_topics = df_rev_new[['business_id','categories']]
df_topics.drop_duplicates(inplace=True)
df_topics['predicted_topics'] = df_topics['business_id'].apply(predict_topics)

C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [37]:
df_topics.head()

,business_id,categories,predicted_topics
0,OQcvO5P3gH0cuJ-bPXwfQQ,"[u'Restaurants', u'Mexican']","[(20, 0.201280702426), (8, 0.111738424262), (0..."
614,VkIKXVbSfcbxybbxbEt98w,"[u'Automotive', u'Car Dealers']","[(7, 0.0841882868644), (26, 0.0797779085732), ..."
820,ckc3l8cSuggloG4NlquEFQ,"[u'Fast Food', u'Restaurants']","[(38, 0.292304696599), (43, 0.145389050797), (..."
856,Du8gPSC88a9OuY1Vk0b52g,"[u'Ice Cream & Frozen Yogurt', u'Food']","[(8, 0.139358053968), (28, 0.12575320667), (26..."
880,y7yo-LAjJnNoYwWX6ZYbqA,"[u'General Dentistry', u'Dentists', u'Health &...","[(24, 0.222685944524), (47, 0.0915175628953), ..."


In [131]:
##df_topics.to_csv(main_path+'topics_top50.csv')

In [38]:
## Assign inferred names to topics

def rename_topics(topic_list):
    return [lda_topic_map[i[0]] for i in topic_list]

In [39]:
df_topics['topic_names'] = df_topics['predicted_topics'].apply(rename_topics)
df_topics.head()

C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,business_id,categories,predicted_topics,topic_names
0,OQcvO5P3gH0cuJ-bPXwfQQ,"[u'Restaurants', u'Mexican']","[(20, 0.201280702426), (8, 0.111738424262), (0...","[mexican, ice cream, service;steak, location, ..."
614,VkIKXVbSfcbxybbxbEt98w,"[u'Automotive', u'Car Dealers']","[(7, 0.0841882868644), (26, 0.0797779085732), ...","[home services, car dealer;auto service;car;au..."
820,ckc3l8cSuggloG4NlquEFQ,"[u'Fast Food', u'Restaurants']","[(38, 0.292304696599), (43, 0.145389050797), (...","[dentist;health, fast food;american, service;c..."
856,Du8gPSC88a9OuY1Vk0b52g,"[u'Ice Cream & Frozen Yogurt', u'Food']","[(8, 0.139358053968), (28, 0.12575320667), (26...","[ice cream, pizza;italian, car dealer;auto ser..."
880,y7yo-LAjJnNoYwWX6ZYbqA,"[u'General Dentistry', u'Dentists', u'Health &...","[(24, 0.222685944524), (47, 0.0915175628953), ...","[vet;health;medical, dinner;food;restaurant, p..."


In [40]:
df_topics.head(3)

,business_id,categories,predicted_topics,topic_names
0,OQcvO5P3gH0cuJ-bPXwfQQ,"[u'Restaurants', u'Mexican']","[(20, 0.201280702426), (8, 0.111738424262), (0...","[mexican, ice cream, service;steak, location, ..."
614,VkIKXVbSfcbxybbxbEt98w,"[u'Automotive', u'Car Dealers']","[(7, 0.0841882868644), (26, 0.0797779085732), ...","[home services, car dealer;auto service;car;au..."
820,ckc3l8cSuggloG4NlquEFQ,"[u'Fast Food', u'Restaurants']","[(38, 0.292304696599), (43, 0.145389050797), (...","[dentist;health, fast food;american, service;c..."


In [41]:
#df_topics.to_csv(main_path+'topics_top50_fin.csv')

In [42]:
## Evaluation
import ast
def check_predictions(actual_cats, pred_topics):
    actual_cats = [j.lower() for j in ast.literal_eval(actual_cats)]
    for i in pred_topics[:3]:
        for j in i.split(";"):
            if j.lower() in actual_cats:
                return 'Yes'
        
    return 'No'

In [43]:
## Calculate prediction accuracy

predict_checks = []
actual_categories = df_topics['categories'].tolist()
predicted_topics = df_topics['topic_names'].tolist()

for i in range(df_topics.shape[0]):
    predict_checks.append(check_predictions(actual_categories[i], predicted_topics[i]))

df_topics['pred_check'] = predict_checks

C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [44]:
#df_topics.to_csv(main_path+'check_sub_predictions_all.csv')

In [58]:
## Accuracy - METHOD I
acc_1 = float(df_topics[df_topics['pred_check']=='Yes'].shape[0])/df_topics.shape[0]
print "Method I accuracy: ",acc_1*100,"%"

Method I accuracy:  33.2026143791 %


In [46]:
##Unique cat list
yelp_cats = [j.lower() for i in df_rev_new['categories'].tolist() for j in ast.literal_eval(i)]
yelp_cats = list(set(yelp_cats))

## METHOD II - Major topics among all reviews of a business


In [47]:
## Predicting topics from all the reviews of a business
def predict_review_topics(noun_tags):
    
    new_review_bow = dictionary.doc2bow(noun_tags)
    new_review_lda = ldamodel[new_review_bow]

    return sorted(new_review_lda,key=itemgetter(1), reverse=True)[:5]

In [48]:
df_topics2 = df_rev_new[['business_id','review_id','noun_tags','categories']]
df_topics2['predicted_topics'] = df_topics2['noun_tags'].apply(predict_review_topics)

C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [49]:
df_topics2.head(3)

,business_id,review_id,noun_tags,categories,predicted_topics
0,OQcvO5P3gH0cuJ-bPXwfQQ,BLIJFaJZ-_fOcBs16fL_6g,"[customer, rating, decor, dicount, food, note,...","[u'Restaurants', u'Mexican']","[(37, 0.738610431108), (40, 0.141389568892)]"
1,OQcvO5P3gH0cuJ-bPXwfQQ,VuKbGklNbOESJSx76_EjyA,"[food, owners, restaurant]","[u'Restaurants', u'Mexican']","[(47, 0.485537400283), (42, 0.274462599717)]"
2,OQcvO5P3gH0cuJ-bPXwfQQ,HBaAmcS9zp5rY1qiMuWygA,"[yummy, meat, service, restaurant, fish, vegas...","[u'Restaurants', u'Mexican']","[(20, 0.8775)]"


In [52]:
## Get the major topics per business
def get_major_topics(bus_id):
    
    df_sub = df_topics2[(df_topics2['business_id'] == bus_id)]
    #print(df_sub).shape[0]
    
    get_topic_list = [j[0] for i in df_sub['predicted_topics'].tolist() for j in i]
    topic_frequency = Counter(get_topic_list)
    
    top_5 = topic_frequency.most_common(5)
    
    return top_5

In [53]:
n4 = datetime.now()

df_topics2_sub = df_topics2[['business_id','categories']]
df_topics2_sub['topic_list'] = df_topics2_sub['business_id'].apply(get_major_topics)
df_topics2_sub.head()

print "Runtime: ",datetime.now()-n4

Runtime:  0:08:04.316000


C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [54]:
df_topics2_sub['topic_names'] = df_topics2_sub['topic_list'].apply(rename_topics)

C:\Users\bhargavic\Anaconda2\envs\gl-env\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [55]:
## Predict the topics per each review of the business

predict_checks2 = []
actual_categories2 = df_topics2_sub['categories'].tolist()
predicted_topics2 = df_topics2_sub['topic_names'].tolist()

for i in range(df_topics2_sub.shape[0]):
    predict_checks2.append(check_predictions(actual_categories2[i], predicted_topics2[i]))

df_topics2_sub['pred_check'] = predict_checks2

In [59]:
## Accuracy - Method II
acc_2 = float(df_topics2_sub[df_topics2_sub['pred_check']=='Yes'].shape[0])/df_topics2_sub.shape[0]
print "Method II accuracy: ",acc_2*100,"%"

Method II accuracy:  51.3647729458 %
